In [1]:
import tweepy,json
access_token="2427460241-yihCbhCrkA6QrS7mhtwkK9FCnoKMvZzPNRFEtYr"
access_token_secret="p6dTQRMwy0SxVP49oHLUXeJ1L4T2cDqPM4RNlcEPErJ9X"
consumer_key="3nHkUhoqNif1x64w2gN7UxfPD"
consumer_secret="SD0qR7yWtUdGDqYxPcUpyNsRcMsX5MJB9z4MxjVKyq6VYwvDp0"
auth= tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)

In [2]:
search_terms = ['home-cooked food for corona patients','homemade food for corona patients','food for corona patients','food for covid patients', 'covid food delivery', 'covid free home cooked food delivery','homemade food covid','homemade food corona','home-cooked food covid','home-cooked food corona']
data = [] 
def stream_tweets(search_term):
    counter = 0 
    for tweet in tweepy.Cursor(api.search, q='\"{}\" -filter:retweets'.format(search_term), count=100, lang='en', tweet_mode='extended').items():
        tweet_details = {}
        tweet_details['time'] = str(tweet.user.created_at)
        tweet_details['id'] = tweet.user.id
        tweet_details['name'] = tweet.user.screen_name
        tweet_details['tweet'] = tweet.full_text
        tweet_details['retweets'] = tweet.retweet_count
        tweet_details['location'] = tweet.user.location
        tweet_details['created'] = tweet.created_at.strftime("%d-%b-%Y")
        tweet_details['followers'] = tweet.user.followers_count
        tweet_details['is_user_verified'] = tweet.user.verified
        data.append(tweet_details)
        counter += 1
        if counter == 1000:
            break
        else:
            pass
    with open('data.json', 'w') as f:
        json.dump(data, f)
    print('done!')

In [3]:
if __name__ == "__main__":
    print('Starting to stream...')
    for k in search_terms:
        stream_tweets(k)
    print('finished!')

Starting to stream...
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
finished!


In [4]:
#converting to csv for simplicity
import pandas as pd
df = pd.read_json('data.json')

df.to_csv('data_tweet.csv',index=None)

In [5]:
import pandas as pd
Data = pd.read_csv('data_tweet.csv',index_col=False)


Data1 = Data.where(pd.notnull(Data), None)

Data1.head()

,time,id,name,tweet,retweets,location,created,followers,is_user_verified
0,2018-09-13 05:41:19,1040113174593163264,Iamsainani,Free food for corona patients in vijayawada \n...,1,Khammam,30-Apr-2021,579,False
1,2012-09-14 14:31:30,823403515,Robojohn11,Ground Report - TamiNadu Govt Hosp's - Food wh...,1,"Chennai, India",28-Apr-2021,233,False
2,2012-05-20 11:54:52,585611074,krishna23tiwari,"The BJP/BJYM begins a doctor's helpline, medic...",0,New Delhi & Mumbai,28-Apr-2021,8,False
3,2020-04-23 10:10:36,1253264950606221312,dhfm_4005,Food for corona patients for free in Vijayawad...,4,None,28-Apr-2021,251,False
4,2020-08-15 07:51:49,1294542222344376322,aashi0309,#COVIDEmergency2021 \nHome made food for Coron...,8,India,27-Apr-2021,568,False


In [6]:
import spacy
from spacy import displacy 
# nlp = spacy.load('en_core_web_sm')

In [7]:
nlp_wk = spacy.load('xx_ent_wiki_sm')

In [8]:
import en_core_web_sm
nlp = en_core_web_sm.load()

In [9]:
import re
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\u2710-\u2900"
#         u"\u2757"
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

In [10]:
lower = lambda x: x.lower()
cap = lambda x: x.title()
bad_chars=['#','!','@']
replace=lambda x:[''.join(i for i in x if not i in bad_chars)]
remove_emoji=lambda x:deEmojify(x)

In [11]:
# df = pd.DataFrame(Data1['tweet'].apply(lower))
# df = pd.DataFrame(Data1['tweet'].apply(cap))
df = pd.DataFrame(Data1['tweet'].apply(replace))
df['tweet']=df['tweet'].str.get(0)
df = pd.DataFrame(df['tweet'].apply(remove_emoji))
df.columns = ['text']

In [12]:
df

,text
0,Free food for corona patients in vijayawada \n...
1,Ground Report - TamiNadu Govt Hosp's - Food wh...
2,"The BJP/BJYM begins a doctor's helpline, medic..."
3,Food for corona patients for free in Vijayawad...
4,COVIDEmergency2021 \nHome made food for Corona...
...,...
282,Free food for covid patients/their families in...
283,indore\n\nFood for Covid Patients https://t.co...
284,Someone has put this google doc with contact i...
285,I want to talk about food delivery. Everyone h...


In [13]:
def extract_named_ents(text):
    return [ent.text for ent in nlp(text).ents if ent.label_ == 'GPE']

def add_named_ents(df):
    df['named_ents'] = df['text'].apply(extract_named_ents)   

In [14]:
add_named_ents(df)

In [15]:
df['named_ents'] = df['named_ents'].str.get(0)
df = df.where(pd.notnull(df), None)

In [16]:
df

,text,named_ents
0,Free food for corona patients in vijayawada \n...,None
1,Ground Report - TamiNadu Govt Hosp's - Food wh...,Kanyakumari
2,"The BJP/BJYM begins a doctor's helpline, medic...",None
3,Food for corona patients for free in Vijayawad...,Vijayawada
4,COVIDEmergency2021 \nHome made food for Corona...,Faridabad
...,...,...
282,Free food for covid patients/their families in...,None
283,indore\n\nFood for Covid Patients https://t.co...,None
284,Someone has put this google doc with contact i...,None
285,I want to talk about food delivery. Everyone h...,None


In [17]:
Data1

,time,id,name,tweet,retweets,location,created,followers,is_user_verified
0,2018-09-13 05:41:19,1040113174593163264,Iamsainani,Free food for corona patients in vijayawada \n...,1,Khammam,30-Apr-2021,579,False
1,2012-09-14 14:31:30,823403515,Robojohn11,Ground Report - TamiNadu Govt Hosp's - Food wh...,1,"Chennai, India",28-Apr-2021,233,False
2,2012-05-20 11:54:52,585611074,krishna23tiwari,"The BJP/BJYM begins a doctor's helpline, medic...",0,New Delhi & Mumbai,28-Apr-2021,8,False
3,2020-04-23 10:10:36,1253264950606221312,dhfm_4005,Food for corona patients for free in Vijayawad...,4,None,28-Apr-2021,251,False
4,2020-08-15 07:51:49,1294542222344376322,aashi0309,#COVIDEmergency2021 \nHome made food for Coron...,8,India,27-Apr-2021,568,False
...,...,...,...,...,...,...,...,...,...
282,2016-11-06 16:10:03,795297203124322305,nisha_sharma17,Free food for covid patients/their families in...,5,None,24-Apr-2021,34,False
283,2021-02-20 16:32:00,1363164427806117896,agri_nirmal,#indore\n\nFood for Covid Patients https://t.c...,2,None,24-Apr-2021,191,False
284,2009-10-28 12:33:31,85807073,thecommonman__,Someone has put this google doc with contact i...,50,யாதும் ஊரே...யாவரும் கேளிர்,26-Apr-2021,70101,False
285,2010-10-13 00:01:16,201946264,gabrielpiemonte,I want to talk about food delivery. Everyone h...,3,"Chicago, IL",25-Apr-2021,2728,False


In [18]:
arr=[
"Andaman and Nicobar Islands"
,"Port Blair"
,"Andhra Pradesh"
,"Adoni"
,"Amaravati"
,"Anantapur"
,"Chandragiri"
,"Chittoor"
,"Dowlaiswaram"
,"Eluru"
,"Guntur"
,"Kadapa"
,"Kakinada"
,"Kurnool"
,"Machilipatnam"
,"Nagarjunakoṇḍa"
,"Rajahmundry"
,"Srikakulam"
,"Tirupati"
,"Vijayawada"
,"Visakhapatnam"
,"Vizianagaram"
,"Yemmiganur"
,"Arunachal Pradesh"
,"Itanagar"
,"Assam"
,"Dhuburi"
,"Dibrugarh"
,"Dispur"
,"Guwahati"
,"Jorhat"
,"Nagaon"
,"Sibsagar"
,"Silchar"
,"Tezpur"
,"Tinsukia"
,"Bihar"
,"Ara"
,"Baruni"
,"Begusarai"
,"Bettiah"
,"Bhagalpur"
,"Bihar Sharif"
,"Bodh Gaya"
,"Buxar"
,"Chapra"
,"Darbhanga"
,"Dehri"
,"Dinapur Nizamat"
,"Gaya"
,"Hajipur"
,"Jamalpur"
,"Katihar"
,"Madhubani"
,"Motihari"
,"Munger"
,"Muzaffarpur"
,"Patna"
,"Purnia"
,"Pusa"
,"Saharsa"
,"Samastipur"
,"Sasaram"
,"Sitamarhi"
,"Siwan"
,"Chandigarh"
,"Chandigarh"
,"Chhattisgarh"
,"Ambikapur"
,"Bhilai"
,"Bilaspur"
,"Dhamtari"
,"Durg"
,"Jagdalpur"
,"Raipur"
,"Rajnandgaon"
,"Dadra and Nagar Haveli"
,"Silvassa"
,"Daman and Diu"
,"Daman"
,"Diu"
,"Delhi"
,"Delhi"
,"New Delhi"
,"Goa"
,"Madgaon"
,"Panaji"
,"Gujarat"
,"Ahmadabad"
,"Amreli"
,"Bharuch"
,"Bhavnagar"
,"Bhuj"
,"Dwarka"
,"Gandhinagar"
,"Godhra"
,"Jamnagar"
,"Junagadh"
,"Kandla"
,"Khambhat"
,"Kheda"
,"Mahesana"
,"Morvi"
,"Nadiad"
,"Navsari"
,"Okha"
,"Palanpur"
,"Patan"
,"Porbandar"
,"Rajkot"
,"Surat"
,"Surendranagar"
,"Valsad"
,"Veraval"
,"Haryana"
,"Ambala"
,"Bhiwani"
,"Chandigarh"
,"Faridabad"
,"Firozpur Jhirka"
,"Gurgaon"
,"Hansi"
,"Hisar"
,"Jind"
,"Kaithal"
,"Karnal"
,"Kurukshetra"
,"Panipat"
,"Pehowa"
,"Rewari"
,"Rohtak"
,"Sirsa"
,"Sonipat"
,"Himachal Pradesh"
,"Bilaspur"
,"Chamba"
,"Dalhousie"
,"Dharmshala"
,"Hamirpur"
,"Kangra"
,"Kullu"
,"Mandi"
,"Nahan"
,"Shimla"
,"Una"
,"Jammu"
,"Kashmir"
,"Anantnag"
,"Baramula"
,"Doda"
,"Gulmarg"
,"Jammu"
,"Kathua"
,"Punch"
,"Rajauri"
,"Srinagar"
,"Udhampur"
,"Jharkhand"
,"Bokaro"
,"Chaibasa"
,"Deoghar"
,"Dhanbad"
,"Dumka"
,"Giridih"
,"Hazaribag"
,"Jamshedpur"
,"Jharia"
,"Rajmahal"
,"Ranchi"
,"Saraikela"
,"Karnataka"
,"Badami"
,"Ballari"
,"Bangalore"
,"Belgavi"
,"Bhadravati"
,"Bidar"
,"Chikkamagaluru"
,"Chitradurga"
,"Davangere"
,"Halebid"
,"Hassan"
,"Hubballi-Dharwad"
,"Kalaburagi"
,"Kolar"
,"Madikeri"
,"Mandya"
,"Mangaluru"
,"Mysuru"
,"Raichur"
,"Shivamogga"
,"Shravanabelagola"
,"Shrirangapattana"
,"Tumkuru"
,"Kerala"
,"Alappuzha"
,"Badagara"
,"Idukki"
,"Kannur"
,"Kochi"
,"Kollam"
,"Kottayam"
,"Kozhikode"
,"Mattancheri"
,"Palakkad"
,"Thalassery"
,"Thiruvananthapuram"
,"Thrissur"
,"Ladakh "
,"Kargil"
,"Leh"
,"Madhya Pradesh"
,"Balaghat"
,"Barwani"
,"Betul"
,"Bharhut"
,"Bhind"
,"Bhojpur"
,"Bhopal"
,"Burhanpur"
,"Chhatarpur"
,"Chhindwara"
,"Damoh"
,"Datia"
,"Dewas"
,"Dhar"
,"Guna"
,"Gwalior"
,"Hoshangabad"
,"Indore"
,"Itarsi"
,"Jabalpur"
,"Jhabua"
,"Khajuraho"
,"Khandwa"
,"Khargon"
,"Maheshwar"
,"Mandla"
,"Mandsaur"
,"Mhow"
,"Morena"
,"Murwara"
,"Narsimhapur"
,"Narsinghgarh"
,"Narwar"
,"Neemuch"
,"Nowgong"
,"Orchha"
,"Panna"
,"Raisen"
,"Rajgarh"
,"Ratlam"
,"Rewa"
,"Sagar"
,"Sarangpur"
,"Satna"
,"Sehore"
,"Seoni"
,"Shahdol"
,"Shajapur"
,"Sheopur"
,"Shivpuri"
,"Ujjain"
,"Vidisha"
,"Maharashtra"
,"Ahmadnagar"
,"Akola"
,"Amravati"
,"Aurangabad"
,"Bhandara"
,"Bhusawal"
,"Bid"
,"Buldana"
,"Chandrapur"
,"Daulatabad"
,"Dhule"
,"Jalgaon"
,"Kalyan"
,"Karli"
,"Kolhapur"
,"Mahabaleshwar"
,"Malegaon"
,"Matheran"
,"Mumbai"
,"Nagpur"
,"Nanded"
,"Nashik"
,"Osmanabad"
,"Pandharpur"
,"Parbhani"
,"Pune"
,"Ratnagiri"
,"Sangli"
,"Satara"
,"Sevagram"
,"Solapur"
,"Thane"
,"Ulhasnagar"
,"Vasai-Virar"
,"Wardha"
,"Yavatmal"
,"Manipur"
,"Imphal"
,"Meghalaya"
,"Cherrapunji"
,"Shillong"
,"Mizoram"
,"Aizawl"
,"Lunglei"
,"Nagaland"
,"Kohima"
,"Mon"
,"Phek"
,"Wokha"
,"Zunheboto"
,"Odisha"
,"Balangir"
,"Baleshwar"
,"Baripada"
,"Bhubaneshwar"
,"Brahmapur"
,"Cuttack"
,"Dhenkanal"
,"Keonjhar"
,"Konark"
,"Koraput"
,"Paradip"
,"Phulabani"
,"Puri"
,"Sambalpur"
,"Udayagiri"
,"Puducherry"
,"Karaikal"
,"Mahe"
,"Puducherry"
,"Yanam"
,"Punjab"
,"Amritsar"
,"Batala"
,"Chandigarh"
,"Faridkot"
,"Firozpur"
,"Gurdaspur"
,"Hoshiarpur"
,"Jalandhar"
,"Kapurthala"
,"Ludhiana"
,"Nabha"
,"Patiala"
,"Rupnagar"
,"Sangrur"
,"Rajasthan"
,"Abu"
,"Ajmer"
,"Alwar"
,"Amer"
,"Barmer"
,"Beawar"
,"Bharatpur"
,"Bhilwara"
,"Bikaner"
,"Bundi"
,"Chittaurgarh"
,"Churu"
,"Dhaulpur"
,"Dungarpur"
,"Ganganagar"
,"Hanumangarh"
,"Jaipur"
,"Jaisalmer"
,"Jalor"
,"Jhalawar"
,"Jhunjhunu"
,"Jodhpur"
,"Kishangarh"
,"Kota"
,"Merta"
,"Nagaur"
,"Nathdwara"
,"Pali"
,"Phalodi"
,"Pushkar"
,"Sawai Madhopur"
,"Shahpura"
,"Sikar"
,"Sirohi"
,"Tonk"
,"Udaipur"
,"Sikkim"
,"Gangtok"
,"Gyalsing"
,"Lachung"
,"Mangan"
,"Tamil Nadu"
,"Arcot"
,"Chengalpattu"
,"Chennai"
,"Chidambaram"
,"Coimbatore"
,"Cuddalore"
,"Dharmapuri"
,"Dindigul"
,"Erode"
,"Kanchipuram"
,"Kanniyakumari"
,"Kodaikanal"
,"Kumbakonam"
,"Madurai"
,"Mamallapuram"
,"Nagappattinam"
,"Nagercoil"
,"Palayankottai"
,"Pudukkottai"
,"Rajapalaiyam"
,"Ramanathapuram"
,"Salem"
,"Thanjavur"
,"Tiruchchirappalli"
,"Tirunelveli"
,"Tiruppur"
,"Tuticorin"
,"Udhagamandalam"
,"Vellore"
,"Telangana"
,"Hyderabad"
,"Karimnagar"
,"Khammam"
,"Mahbubnagar"
,"Nizamabad"
,"Sangareddi"
,"Warangal"
,"Tripura"
,"Agartala"
,"Uttar Pradesh"
,"Agra"
,"Aligarh"
,"Amroha"
,"Ayodhya"
,"Azamgarh"
,"Bahraich"
,"Ballia"
,"Banda"
,"Bara Banki"
,"Bareilly"
,"Basti"
,"Bijnor"
,"Bithur"
,"Budaun"
,"Bulandshahr"
,"Deoria"
,"Etah"
,"Etawah"
,"Faizabad"
,"Farrukhabad"
,"Fatehgarh"
,"Fatehpur"
,"Fatehpur Sikri"
,"Ghaziabad"
,"Ghazipur"
,"Gonda"
,"Gorakhpur"
,"Hamirpur"
,"Hardoi"
,"Hathras"
,"Jalaun"
,"Jaunpur"
,"Jhansi"
,"Kannauj"
,"Kanpur"
,"Lakhimpur"
,"Lalitpur"
,"Lucknow"
,"Mainpuri"
,"Mathura"
,"Meerut"
,"Mirzapur"
,"Vindhyachal"
,"Moradabad"
,"Muzaffarnagar"
,"Partapgarh"
,"Pilibhit"
,"Prayagraj"
,"Rae Bareli"
,"Rampur"
,"Saharanpur"
,"Sambhal"
,"Shahjahanpur"
,"Sitapur"
,"Sultanpur"
,"Tehri"
,"Varanasi"
,"Uttarakhand"
,"Almora"
,"Dehra Dun"
,"Haridwar"
,"Mussoorie"
,"Nainital"
,"Pithoragarh"
,"West Bengal"
,"Alipore"
,"Alipur Duar"
,"Asansol"
,"Baharampur"
,"Bally"
,"Balurghat"
,"Bankura"
,"Baranagar"
,"Barasat"
,"Barrackpore"
,"Basirhat"
,"Bhatpara"
,"Bishnupur"
,"Budge Budge"
,"Burdwan"
,"Chandernagore"
,"Darjiling"
,"Diamond Harbour"
,"Dum Dum"
,"Durgapur"
,"Halisahar"
,"Haora"
,"Hugli"
,"Ingraj Bazar"
,"Jalpaiguri"
,"Kalimpong"
,"Kamarhati"
,"Kanchrapara"
,"Kharagpur"
,"Koch Bihar"
,"Kolkata"
,"Krishnanagar"
,"Malda"
,"Midnapore"
,"Murshidabad"
,"Navadwip"
,"Palashi"
,"Panihati"
,"Purulia"
,"Raiganj"
,"Santipur"
,"Shantiniketan"
,"Shrirampur"
,"Siliguri"
,"Siuri"
,"Tamluk"
]

In [19]:
df['text']=df['text'].apply(cap)
for i in range(0,len(df)):
    if df['named_ents'][i]==None:
        for k in arr:
            if k in df['text'][i]:
                df['named_ents'][i]=k
    elif df['named_ents'][i]!=None:
        for k in arr:
            if k in df['text'][i]and k not in df['named_ents'][i]:
                df['named_ents'][i]+=','+k
        

In [20]:
df

,text,named_ents
0,Free Food For Corona Patients In Vijayawada \n...,Vijayawada
1,Ground Report - Taminadu Govt Hosp'S - Food Wh...,Kanyakumari
2,"The Bjp/Bjym Begins A Doctor'S Helpline, Medic...",None
3,Food For Corona Patients For Free In Vijayawad...,Vijayawada
4,Covidemergency2021 \nHome Made Food For Corona...,"Faridabad,Haryana"
...,...,...
282,Free Food For Covid Patients/Their Families In...,Bhilwara
283,Indore\n\nFood For Covid Patients Https://T.Co...,Indore
284,Someone Has Put This Google Doc With Contact I...,Coimbatore
285,I Want To Talk About Food Delivery. Everyone H...,None


In [21]:
Data1['tweet_location']=df['named_ents']

In [22]:
# export_excel = Data1.to_excel (r'C:\Users\Khushboo\Desktop\Cargill\sample.xlsx', index = None, header=True)

In [23]:
# export_excel = df.to_excel (r'C:\Users\Khushboo\Desktop\Cargill\sample1.xlsx', index = None, header=True)

In [24]:
#creating connection to mysql and creating db
import mysql.connector as msql
from mysql.connector import Error
try:
    conn = msql.connect(host='sql129.main-hosting.eu', user='u291509283_cargill',  
                        password='Cargill123')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("CREATE DATABASE u291509283_cargill")
        print("Tweet_DB database is created")
except Error as e:
    print("Error while connecting to MySQL", e)

Error while connecting to MySQL 1007 (HY000): Can't create database 'u291509283_cargill'; database exists


In [26]:
#updation in database
try:
    conn = msql.connect(host='sql129.main-hosting.eu', 
                           database='u291509283_cargill', user='u291509283_cargill', 
                           password='Cargill123')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        cursor.execute('DROP TABLE IF EXISTS Tweet_data;')
        print('Creating table....')
        cursor.execute("CREATE TABLE Tweet_data (time DATETIME,user_id BIGINT,name VARCHAR(255), tweet VARCHAR(255) ,retweets INTEGER,location VARCHAR(255),created VARCHAR(255),followers INTEGER,is_user_verified VARCHAR(255),tweet_location VARCHAR(255))")
        print("Tweet table is created....")
        for i,row in Data1.iterrows():
            sql = "INSERT INTO u291509283_cargill.Tweet_data VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            print("Record inserted")
            
            conn.commit()
        
except Error as e:
    print("Error while connecting to MySQL", e)

You're connected to database:  ('u291509283_cargill',)
Creating table....
Tweet table is created....
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Rec